# **STUDENT AI** - EDA
Exploratory Data Analysis

## Objectives

Inspect the dataset and solve any issues that might arise from wrong data types, or missing / wrong values

## Inputs

Continues to assess dataset loaded in previous notebook.

## Outputs

Saves the cleaned dataset back to inputs/dataset folder for futher use


---

# Import required libraries

In [1]:
import os
import pandas as pd

print('All Libraries Loaded')

All Libraries Loaded


# Change working directory

### Set the working directory to notebook parent folder
If the output does not match, click **'clear all outputs'** and then **'restart'** the notebook. 
Then run cells from top to bottom.

In [2]:
current_dir = os.getcwd()
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
print('If correct, Active Directory should read: /workspace/student-AI')
print(f"Active Directory: {current_dir}")

If correct, Active Directory should read: /workspace/student-AI
Active Directory: /workspace/student-AI


### Load cleaned dataset

In [3]:
df = pd.read_csv(f"outputs/dataset/Expanded_data_with_more_features_clean.csv")
df.head()

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
1,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
2,female,group B,associate's degree,standard,none,married,regularly,yes,1.0,school_bus,5 - 10,73,84,79
3,female,group B,some college,standard,completed,widowed,never,no,1.0,private,5 - 10,85,93,89
4,male,group B,some college,free/reduced,none,married,sometimes,yes,1.0,private,> 10,41,43,39


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19243 entries, 0 to 19242
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Gender               19243 non-null  object 
 1   EthnicGroup          19243 non-null  object 
 2   ParentEduc           19243 non-null  object 
 3   LunchType            19243 non-null  object 
 4   TestPrep             19243 non-null  object 
 5   ParentMaritalStatus  19243 non-null  object 
 6   PracticeSport        19243 non-null  object 
 7   IsFirstChild         19243 non-null  object 
 8   NrSiblings           19243 non-null  float64
 9   TransportMeans       19243 non-null  object 
 10  WklyStudyHours       19243 non-null  object 
 11  MathScore            19243 non-null  int64  
 12  ReadingScore         19243 non-null  int64  
 13  WritingScore         19243 non-null  int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 2.1+ MB
